# Matrix Factorization

In a purely mathematical sense, given a matrix $R$, the objective of `matrix factorization` is to find two `factor matrices` to reconstruct / approximate $R$ as $\hat{R} = P^T Q$ where $R$ is $m \times n$, $P$ is $m \times f$, $Q$ is $n \times f$ and $f$ is the hyperparameter which chooses the dimensionality of the approximation.



# Maximum Likelihood

While $R$, $P$ and $Q$ are the standard letter names for the mathematical problem, in statistics the names are less standard. I will now start using a notation that is more useful/common but is nonetheless highly variable within the machine learning community, and note that $R$ is usually a `rating matrix`, viz.

$$R = X^T Y$$

Model the observed matrix $R$ as being drawn from normally distributed ratings as

$$R_{u,i} \sim \mathcal{N}(X_u^TY_i, \lambda^{-1}_R)$$

where the normal distribution has mean $\mu=X_u^TY_i$ and variance $\sigma^2 = \lambda^{-1}_r$.

and factor matrices are also normally distributed with mean zero and variance $\lambda^{-1}_X$ and $\lambda^{-1}_Y$ along $f$ dimensions,

$$X_u \sim \mathcal{N}(0, \lambda_X^{-1}I_f)$$

$$Y_i \sim \mathcal{N}(0, \lambda_Y^{-1}I_f)$$

This would give a likelihood $\mathcal{L}$ as

$$\mathcal{L} = \prod_{u,i} \mathcal{N}(X_u^TY_i, \lambda^{-1}_r) \mathcal{N}(0, \lambda_X^{-1}I_f)   \mathcal{N}(0, \lambda_Y^{-1}I_f). $$

Since it is easier to work with a sum than a product and the log is convex, we can form the log-likelihood as

$$logL = \frac{\lambda_p}{2}\sum_{u,i} (r_{ui} - X_u^TY_i)^2 + \frac{\lambda_X}{2}\sum_u \|X_u\|^2 + \frac{\lambda_Y}{2}\sum_i \|Y_i\|^2$$

and its extremum values will be the same as the likelihood. Depending on whether you are from the stats camp and want to maximize the $logL$ or from the ML camp and want to minimize $-logL$, either way you have to take the derivative.

# Implicit Data and Implicit Matrix Factorization

Historically speaking, early problems in recommender systems tried to predict a `rating` a user $u$ would give to item $i$.

More recently, that approach has fallen out of favor with an emphasis shifting to `preference` prediction; concretely we now aim to predict the preference and/or future consumption of user $u$ for/of item $i$.

This shift also highlights the move away from `explicit data` toward `implicit data` because of a few key advantages --

* Implicit data is usually more plentiful and is already naturally collected by many services, e.g. at Staples we already know what people purchase, regardless of if they tell us a rating of that product.

* Ratings are more noisy because people may change how they rate items based on mood, time of day, personal bias to rate higher or lower than other users, etc.

* Predicting a preference might more closely match the end goal of a system/product since even if you correctly identify how a user may rate an item, there is no guarantee the user will consume that item in the first place.

In practice we observe a matrix $C$, often called the `click matrix` or also the `consumption matrix`.

The goal of `implicit matrix factorization` is to reconstruct the binary preference matrix $P$.

Based on the observed values (non-zero) and unobserved values (zero) of $C$, we create the binary preference matrix $P$. 

$$p_{ui} = \begin{cases}
0 &\quad c_{ui} = 0\\
1 &\quad c_{ui} > 0
\end{cases}
$$

We use $P$ as the ground truth in training and evaluation throughout.

# Weighted Matrix Factorization (WMF)

Weighted Matrix factorization differs from the formulation outlined at the top by adding a weight value $w_{ui}$ associated with each preference prediction $p_{ui}$. This addition is unorthodox in the maximum likelihood formulation and it's not possible to construct the likelihood as we did above in the same way. From here, we will treat this weighting factor as a simple added heuristic and express the weighted matrix factorization cost function --

$$J = \sum_{u,i} w_{ui}\big( p_{ui} - X_u^TY_i\big)^2 + \lambda_X \sum_u \|X_u\|^2 + \lambda_Y\sum_i \|Y_i\|^2$$

To minimize the cost function we take the derivative with respect to the two factor matrices

$$0 = \frac{\partial J}{\partial X_v} = \sum_{u,i} -2 w_{ui} \big(p_{ui} - X_u^TY_i\big)Y_i\frac{\partial X_u}{\partial X_v} + 2\lambda_X \sum_u X_u \frac{\partial X_u}{\partial X_v}$$

$$0 = \frac{\partial J}{\partial Y_j} = \sum_{u,i} -2 w_{ui} \big(p_{ui} - X_u^TY_i\big)X_u\frac{\partial Y_i}{\partial Y_j} + 2\lambda_Y \sum_i Y_i \frac{\partial Y_i}{\partial Y_j}$$



# Alternating Least Squares Regression (ALS)

The set of derivatives $\frac{\partial J}{\partial X_v}$ and $\frac{\partial J}{\partial Y_j}$ above create a non-convex problem, but taking each derivative separately, the problem is convex and has a closed form solution.

The alternating least squares (ALS) approximation attempts to find the optimal parameters to the WMF cost function by alternatively performing closed formed updates to the $X_u$ and the $Y_i$, often referred to as user and item updates.

Because the equations for the optimal $X_u$ and $Y_i$ are symmetric, we will treat just a single equation below, namely, the equation for $X_u$. We recover the results for $Y_i$ by swapping $X \leftrightarrow Y$ and index $u \leftrightarrow i$ in all the following results.

Additionally, the sparsity of the observed click matrix $C$ can provide substantial computational speed up if properly exploited during computation.

# User Update Equation

$$0 = \sum_{u,i} - w_{ui} \big(p_{ui} - X_u^TY_i\big)Y_i\frac{\partial X_u}{\partial X_v} + \lambda_X \sum_u X_u \frac{\partial X_u}{\partial X_v}$$

$$\frac{\partial X_u}{\partial X_v} = \delta_{u,v}$$

$$0 = -\sum_i \big[w_{ui} \big(p_{ui} - X_u^TY_i\big)Y_i\big] + \lambda_X X_u$$

$$\sum_i w_{ui} X_uY_i^TY_i + \lambda_X X_u = \sum_i w_{ui} p_{ui}Y_i$$

$$\Big(\sum_i w_{ui} Y_i^TY_i + \lambda_X I \Big) X_u = \sum_i w_{ui} p_{ui}Y_i$$

We can also right this in matrix form, i.e. rather than find the contribution from each item $i$ in the sum, we can find the contribution from all items for this user simultaneously,

$$ \big(Y^TW^uY + \lambda I_f \big) X_u = Y^TW^up_u$$

where $W^u$ is a $n \times n$ square item matrix with the weights along the diagonal and $p_u$ is a n-dimensional vector with the user preference score for each item.


# Sparse and Fast Updates

Naive computation of the above update would be slow due to the all item matrix products in $Y^TW^uY$.

Although in the end it will be optimal to perform vectorized updates with the entire matrix $Y$, for illustrating how we optimize for fast updates it is best to return to the sum on each item $i$.

The goal is to only sum over the observed values for each user update and take advantage of the sparsity in the consumption matrix.  Additionally, at some point we want to weight observed and unobserved values differently, and we can proceed here without knowing any specifics around how we will choose those different weights. In that spirit, we will split the sums between the values, $c_{ui} = 0$ and $c_{ui} > 0$ and assigned the following different weights:

$$w_{ui} = \begin{cases}
w_0 &\quad c_{ui} = 0\\
w_1 &\quad c_{ui} > 0
\end{cases}
$$

$$\Big(\sum_{i \in c_{ui} = 0} w_0 Y_i^TY_i + \sum_{i\in c_{ui} > 0} w_1 Y_i^TY_i + \lambda_X I_f \Big) X_u = \sum_{i \in c_{ui} = 0} w_0 p_{ui}Y_i + \sum_{i \in c_{ui} > 0} w_1 p_{ui}Y_i$$

We work on the left side first and express the sum on unobserved values in terms of the sum on all values minus the sum on observed values.

$$ \sum_{i \in c_{ui} = 0}w_0 Y_i^TY_i = \sum_i w_0 Y_i^TY_i - \sum_{i \in c_{ui} > 0}w_0 Y_i^TY_i$$

On the left side, substituting in and combining the sum on observed values, we arrive at the left sum being independent of the user $u$ and right sum on observed values only, taking advantage of the sparsity of $C$.

$$\Big(\sum_i w_0 Y_i^TY_i + \sum_{i\in c_{ui} > 0} (w_1-w_0) Y_i^TY_i + \lambda_X I \Big) X_u = \sum_{i \in c_{ui} = 0} w_0 p_{ui}Y_i + \sum_{i \in c_{ui} > 0} w_1 p_{ui}Y_i$$

For the right side, we note that all terms directly multiply the preference matrix $P$. In the case of the sum on unobserved values, $p_{ui}=0$ by construction, and  the entire sum is zero.

$$\Big(\sum_i w_0 Y_i^TY_i + \sum_{i\in c_{ui} > 0} (w_1-w_0) Y_i^TY_i + \lambda_X I \Big) X_u = \sum_{i \in c_{ui} > 0} w_1 p_{ui}Y_i$$


# Real Scenario

Let's take the following real scenario make one final simplification -

$$w_{ui} = 1 + \alpha c_{ui}$$

This choice for $w_{ui}$ corresponds to the above choices for $w_0$ and $w_1$:

$$\Big(\sum_i Y_i^TY_i + \sum_{i\in c_{ui} > 0} \alpha c_{ui} Y_i^TY_i + \lambda_X I_f \Big) X_u = \sum_{i \in c_{ui} > 0} (1+\alpha c_{ui}) p_{ui}Y_i$$

Finally, going back to the vectorized matrix update,

$$ \big(Y^TY + \alpha Y^TC^uY + \lambda_X I_f \big)X_u = Y^T (I_n + \alpha C^u)p_u$$

Dimensions check
$$(f x n) (n x f) + (f x n)(n x n)(n x f) + (f x f)] (f x 1) = (f x n) (n x n) (n x 1)$$

It is useful to define and precompute $S^u = \alpha C^u$ since they appear together.

In [3]:
def linear_surplus_confidence_matrix(C, alpha):
    # To construct the surplus confidence matrix, we need to operate only on
    # the nonzero elements.
    # This is not possible: S = alpha * C
    S = C.copy()
    S.data = alpha * S.data
    return S

In [4]:
def recompute_factors_full(Y, S, lambda_reg):
    """
    Written from the point of view of updating the user factors X with
    fixed item vectors Y.
    """
    m = S.shape[0]  # m = number of users
    f = Y.shape[1]  # f = number of factors
    YTY = np.dot(Y.T, Y)  # precompute this
    YTYpR = YTY + lambda_reg * np.eye(f)
    A_stack = np.empty((m, f), dtype='float32')
    B_stack = np.empty((m, f, f), dtype='float32')

    for u in xrange(m):
        s_u, idx_u = get_row(S, u)
        # i_u indexes the nz item elements for user u
        Y_u = Y[idx_u]  # exploit sparsity
        A = (s_u + 1).dot(Y_u)
        YTSY = np.dot(Y_u.T, (Y_u * s_u[:, None]))
        B = YTSY + YTYpR
        A_stack[u] = A
        B_stack[u] = B
    X_stack = solve_sequential(A_stack, B_stack)
    
    return X_stack

def get_row(S, i):
    """
    Args:
        S: scipy.sparse.csr_matrix
        i: index to the ith row in S
        
    Returns:
        S.data[lo:hi]: the non-zero elements in row i of S
        S.indices[lo:hi]: the column indices of the non-zero elements in row i of S
    """
    lo, hi = S.indptr[i], S.indptr[i + 1]
    return S.data[lo:hi], S.indices[lo:hi]

def solve_sequential(As, Bs):
    """
    """
    X_stack = np.empty_like(As, dtype=As.dtype)
    for k in xrange(As.shape[0]):
        X_stack[k] = np.linalg.solve(Bs[k], As[k])
    return X_stack

In [5]:
# summarize WMF_ML20M
num_factors = 100
num_iters = 5
lambda_X_reg = 1e-5
lambda_Y_reg = 1e-5
alpha = 10
init_std=0.01

S = linear_surplus_confidence_matrix(train_data, alpha=alpha)

num_users, num_items = S.shape

ST = S.T.tocsr()

random_state=98765
if type(random_state) is int:
    np.random.seed(random_state)
elif random_state is not None:
    np.random.setstate(random_state)
    
# T: transpose
# u: user index
# i: item index

# f: number of latent factors

# S: surplus click matrix

# X: user factors
# Y: item factors

Y = np.random.randn(num_items, num_factors).astype('float32') * init_std

old_ndcg = -np.inf
for i in xrange(num_iters):

    X = recompute_factors_full(Y, S, lambda_X_reg)
    
    Y = recompute_factors_full(X, ST, lambda_Y_reg)

    vad_ndcg = rec_eval.normalized_dcg_at_k(S, vad_data, X, Y, k=100, batch_users=5000)

    if old_ndcg > vad_ndcg:
        break
    old_ndcg = vad_ndcg
    X_best = X.copy()
    Y_best = Y.copy()

NameError: name 'train_data' is not defined